In [1]:
import os

# set the current working directory
curr_path = os.getcwd().split("/")[:-1]
curr_path = "/".join(curr_path)
os.chdir(curr_path)

In [2]:
import numpy as np
# read dataset
from pytspl import load_dataset

sc, _, flow = load_dataset("forex")
# get the y from flow dict
y = np.fromiter(flow.values(), dtype=float)

print("Flow:", len(y))

Generating coordinates using spring layout.
Num. of nodes: 25
Num. of edges: 210
Num. of triangles: 770
Shape: (25, 210, 770)
Max Dimension: 2
Coordinates: 25
Flow: 210
Flow: 210


In [3]:
from pytspl.hogde_gp import HodgeGPTrainer

# create the trainer object
hogde_gp = HodgeGPTrainer(sc = sc, y = y)

In [4]:
# set the parameters

# set the training ratio
train_ratio = 0.2
# set the data normalization
data_normalization = False

# get the eigenpairs
eigenpairs = hogde_gp.get_eigenpairs()

# split the data into training and testing sets
x_train, y_train, x_test, y_test, x, y  = hogde_gp.train_test_split(train_ratio=train_ratio, data_normalization=data_normalization)

x_train: (42,)
x_test: (168,)
y_train: (42,)
y_test: (168,)


In [5]:
x_train[0], y_train[0]

(tensor(34.), tensor(4.4740))

In [6]:
from pytspl.hogde_gp.kernel_serializer import KernelSerializer

# set the kernel parameters

kernel_type = "matern" # kernel type
data_name = "forex" # data set name

# serialize the kernel
kernel = KernelSerializer().serialize(eigenpairs=eigenpairs, kernel_type=kernel_type, data_name=data_name)

In [7]:
import gpytorch
from pytspl.hogde_gp import ExactGPModel

likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood, kernel, mean_function=None)

In [8]:
type(kernel)

pytspl.hogde_gp.forex.matern.MaternKernelForex

In [9]:
import torch
output_device = "cpu"

if torch.cuda.is_available():
   model = model.to(output_device)
   likelihood = likelihood.to(output_device)

In [10]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:50} value = {param.item()}')

Parameter name: likelihood.noise_covar.raw_noise                   value = 0.0
Parameter name: mean_module.raw_constant                           value = 0.0
Parameter name: covar_module.raw_outputscale                       value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa_down            value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa_up              value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa                 value = 0.0
Parameter name: covar_module.base_kernel.raw_mu                    value = 0.0
Parameter name: covar_module.base_kernel.raw_mu_down               value = 0.0
Parameter name: covar_module.base_kernel.raw_mu_up                 value = 0.0
Parameter name: covar_module.base_kernel.raw_h                     value = 0.0
Parameter name: covar_module.base_kernel.raw_h_down                value = 0.0
Parameter name: covar_module.base_kernel.raw_h_up                  value = 0.0


In [11]:
# train the model
model.train()
likelihood.train()
hogde_gp.train(model, likelihood, x_train, y_train)

Iteration 1/1000 - Loss: 3.867 
Iteration 2/1000 - Loss: 3.571 
Iteration 3/1000 - Loss: 3.321 
Iteration 4/1000 - Loss: 3.112 
Iteration 5/1000 - Loss: 2.939 
Iteration 6/1000 - Loss: 2.795 
Iteration 7/1000 - Loss: 2.676 
Iteration 8/1000 - Loss: 2.577 
Iteration 9/1000 - Loss: 2.495 
Iteration 10/1000 - Loss: 2.426 
Iteration 11/1000 - Loss: 2.369 
Iteration 12/1000 - Loss: 2.322 
Iteration 13/1000 - Loss: 2.283 
Iteration 14/1000 - Loss: 2.251 
Iteration 15/1000 - Loss: 2.226 
Iteration 16/1000 - Loss: 2.206 
Iteration 17/1000 - Loss: 2.190 
Iteration 18/1000 - Loss: 2.179 
Iteration 19/1000 - Loss: 2.170 
Iteration 20/1000 - Loss: 2.164 
Iteration 21/1000 - Loss: 2.160 
Iteration 22/1000 - Loss: 2.157 
Iteration 23/1000 - Loss: 2.156 
Iteration 24/1000 - Loss: 2.155 
Iteration 25/1000 - Loss: 2.155 
Iteration 26/1000 - Loss: 2.155 
Iteration 27/1000 - Loss: 2.155 
Iteration 28/1000 - Loss: 2.155 
Iteration 29/1000 - Loss: 2.155 
Iteration 30/1000 - Loss: 2.155 
Iteration 31/1000 -

In [12]:
# evaluate the model
hogde_gp.predict(model, likelihood, x_test, y_test)

Test MAE: 0.012508535757660866
Test MSE: 0.001972610130906105
Test R2: 0.9996838569641113
Test MLSS: -3.0074620246887207
Test NLPD: -3.514639377593994


MultivariateNormal(loc: torch.Size([168]))